# Project: Non-Thermal Emission from the Pulsar Wind Nebula N 157B
In this project you will build the model as shown on Figure 2 of this paper:
[https://www.aanda.org/articles/aa/full_html/2012/09/aa19906-12/aa19906-12.html]

This notebook has 9 exercises for a total of 23 points. 

## Imports

First of all, we need to import a few modules:

In [ ]:
import astropy.units as u

import numpy as np

import matplotlib.pyplot as plt

from naima.models import Synchrotron, InverseCompton, PionDecay
from naima.models import PowerLaw, BrokenPowerLaw, TableModel, ExponentialCutoffPowerLaw

## Parameter Definition
First we will define several parameters to be used in the following. You should find all these parameters in the publication. The paper discusses two different models: the main model with a cooling break (green lines in Figure 2) and an alternative model (red line). You need only one, you can choose your preferred model.

### Exercise
Choose your model. Write in the following text box which model you will use.

**[1 mark]**

write your choice here

### Exercise
Next you have to find the corresponding parameters in the publication. Put the values in the next box. 
E_1 is the intrinsic break and E_2 is the cooling break (or second break). 
The spectral indices are p_1 for the radio part, p_2 for the uncooled spectrum and p_3 for the cooled spectrum. Note that the paper text list the indices in a different order, p_3 should always be larger than p_2.
We is the total energy in electrons.
B is the magnetic field.

**[1 mark]**

In [ ]:
E_1 = 10 * u.GeV    # you need to change this value
E_2 = 1 * u.TeV     # you need to change this value

p_1 = 1.            #  you need to change this value
p_2 = 2.            #  you need to change this value
p_3 = 3.            #  you need to change this value

We = 1e49*u.erg     #  you need to change this value

B = 50e-6*u.G        #  you need to change this value

E_min and E_max are the minimal and maximal energy for the electrons. This is given here, no need to change the values.

In [ ]:
E_min = 511 * u.keV
E_max = 1e15 *u.eV

We will also need a range of photon energies which we will need to plot.

In [ ]:
photon_energies = np.logspace(-18, 2, 200) * u.TeV

We set also the distance to the value used in the paper:

In [ ]:
d = 48*u.kpc

## Building a Model for the Electron Population
In the lecture we have discussed a straight power law and a power law with an exponential cut-off. Looking at the naima documentation (https://naima.readthedocs.io/en/latest/api-models.html) you will find that there are further models, including a broken power law. A broken power law are just two power laws connecting at the break energy. But for N 157B we need a model with two breaks. Here you will get some instructions how to implement such a model.

We also need a value for the amplitude. We can use any value. The amplitude will change anyway when we set the total energy.

In [ ]:
A = 1./u.TeV

Let's start with a broken power law and build the first two parts of our spectrum.

In [ ]:
model1 = BrokenPowerLaw(amplitude = A, e_0 = 1*u.TeV, e_break = E_1, alpha_1 = p_1, alpha_2 = p_2)

This function should describe our model for energies in the range E_min to the second break, E_2. Let's define an energy range for this:

In [ ]:
energies1 = np.logspace(np.log10(E_min.to(u.TeV).value), np.log10(E_2.to(u.TeV).value), 100) *u.TeV

Let's make a plot of what we have:

In [ ]:
plt.loglog(energies1, model1(energies1))

Now we need a simple power law, starting at E_2 and going up to E_max. But it is important that the two model parts match at E_2, i.e. the value of model1 at E_2 is the same as the value of model2 at E_2. We will achieve this by using E_2 as reference energy and the value of model1 at E_2 as amplitude.

In [ ]:
model2 = PowerLaw(amplitude = model1(E_2), e_0 = E_2, alpha = p_3)

This part of the model is defined from E_2 to E_max. Let's define a second energy range:

In [ ]:
energies2 = np.logspace(np.log10(E_2.to(u.TeV).value), np.log10(E_max.to(u.TeV).value), 100) *u.TeV

Now we make a plot of both components to see if all works well.

In [ ]:
plt.loglog(energies1, model1(energies1), label = 'model 1')
plt.loglog(energies2, model2(energies2), label = 'model 2')

plt.legend()

Good, this worked. Now we will create two arrays, one for the energies and one for corresponding particle numbers. We achieve this by joining the two arrays we already have. But the last value of model1 and the first value of model2 are exactly the same, we do not want this value twice in our new array. So we start the second part at the second entry which is achieved with [1:] (One means we start with the second entry, the first entry is number 0. No value after the colon means that we go all the way to the end.)

In [ ]:
energies = np.hstack([energies1, energies2[1:]])
values = np.hstack([model1(energies1), model2(energies2[1:])])

Finally we create a TableModel with the values we have.

In [ ]:
model_e = TableModel(energies, values)

Let's plot again, just to be sure:

In [ ]:
plt.loglog(energies, model_e(energies))

## Synchrotron Emission
We will start to build the synchrotron emission model.

### Exercise
Make the following cell to work. You need to insert the electron model and the magnetic field.

**[1 mark]**

In [ ]:
sync = Synchrotron(Eemin = E_min, Eemax = E_max)

Now we need to set the total energy of the electrons:

In [ ]:
sync.set_We(We)

### Exercise
Get the SED of the synchortron emission. Do not forget the distance.

**[1 mark]**

In [ ]:
sync_sed = # your code here

In the following we make a plot of the emission. Note the last lines which set the axes labels and limits. You will need to copy these lines when you make the plots yourself.

In [ ]:
plt.loglog(photon_energies, sync_sed)

# keep these lines
plt.ylim(1e-14,1e-10)

plt.ylabel('$E^2 dN/dE$ [{}]'.format(sync_sed.unit))
plt.xlabel('E [{}]'.format(photon_energies.unit))

## Inverse Compton Emission
Now we build the model for the Inverse Compton emission. Note that there are three different photon fields to be considered. Let's start with the IC off the Cosmic Microwave Background. This is easy as the CMB is already defined:

In [ ]:
IC_1 = InverseCompton(model_e, ['CMB'], Eemin = E_min, Eemax = E_max)

Next we build the model for IC emission off the infrared emission from LH99. It has a temperature of 80 K and an energy density 8.9 eV/cm3. Note where these values go in the following.

In [ ]:
IC_2 = InverseCompton(model_e, [['LH 99', 80*u.K, 8.9*u.eV/u.cm**3]], Eemin = E_min, Eemax = E_max)

### Exercise
Build the third IC model for the infrared emission from 30 Doradus.

**[2 marks]**

In [ ]:
IC_3 = # your code here

### Exercise
Make one plot containing all three models.

**[3 marks]**

In [ ]:
# your code here


# keep these lines
plt.ylim(1e-14,1e-10)

plt.ylabel('$E^2 dN/dE$ [{}]'.format(sync_sed.unit))
plt.xlabel('E [{}]'.format(photon_energies.unit))

Expected output:



## Hadronic Emission
Now we build the model for the hadronic emission (blue line in Figure 2 of the paper).

### Exercise
We need a model for the protons. Build a power law model with exponential cut-off using the parameters for the proton spectrum from the paper.

**[2 marks]**

In [ ]:
model_p = # your code here

### Exercise
Now build a hadronic model for the emission from these protons. Set the ambient density and the total energy of the protons. Make sure that you pair the correct total energy with the mathing density. For instance the mentioned 100 particles/cm3 is for a single supernova explosion, this energy is not given in the paper. If your plot does not reach the same level as in the paper then you probably have mixed up the density and the corresponding total energy.

**[3 marks]**

In [ ]:
#had = # your code here

### Exercise
Make a plot of the emission.

**[3 marks]**

In [ ]:
# your code here

Expected output:



## Final Plot
Combine all models in one plot. Make sure that you have labels and units on the axes and that you have a legend for all models.

**[6 marks]**

In [ ]:
# your code here

Expected output:



## Submission

Before you submit your work you should make a few checks that everything works fine.

1. Save your notebook as a PDF (File->Download As->PDF). This document will help you debugging in the next step.
1. If PDF export does not work: You can do File->Print Preview and then print to a file.
1. Restart the kernel and rerun the entire notebook (Kernel->Restart & Run All). This will delete all variables (but not your code) and rerun the notebook in one go. If this does not go through the endthen you have to fix it. You will see at which cell the run stopped. A common mistake is using a variable that is defined only at a later stage.
1. You think you fixed everything? Redo step 2 (Kernel->Restart & Run All)

You have to download and submit 2 files, the jupyter notebook and a pdf.
- Jupyter notebook. File->Download As->Notebook (.ipynb). Save this file on your disk.
- PDF file. File->Download As->PDF. Save this file on your disk.
- If PDF export does not work. You can do File->Print Preview and then print to a file.

Please submit the two files on Ulwazi.